In [28]:
import json
import re
import pandas as pd
import unicodedata

In [ ]:
with open("data/professores.json", encoding="utf-8") as f:
    professores = json.load(f)


def tem_formacao(prof, nivel):
    for f in prof.get("formacao_academica", []):
        desc = f.get("tipo_formacao", "").lower()
        if nivel in desc:
            return "sim"

    return "não"


def calcular_anos_experiencia(prof):
    anos = []
    for f in prof.get("formacao_academica", []):
        tipo = f.get("tipo_formacao", "").lower()
        # Procura por graduação
        if "graduação" in tipo or "graduacao" in tipo:
            # Tenta pegar o ano de conclusão
            ano = f.get("ano_conclusao")
            if ano and str(ano).isdigit():
                anos.append(int(ano))
            else:
                # Alternativamente, tenta extrair do campo 'periodo'
                periodo = f.get("periodo", "")git 
                if " - " in periodo:
                    partes = periodo.split(" - ")
                    if len(partes) > 1 and partes[1].isdigit():
                        anos.append(int(partes[1]))
    if anos:
        return 2025 - min(anos)
    return None


dados = []

for prof in professores:
    nome = prof.get("identificacao", {}).get("nome", "")
    mestrado = tem_formacao(prof, "mestrado")
    doutorado = tem_formacao(prof, "doutorado")
    pos_doutorado = "sim" if prof.get("pos_doutorado") else "não"
    producoes = prof.get("producao_bibliografica", [])
    quantidade_producoes = len(producoes)
    coautores = prof.get("coautores_publicacoes", [])
    quantidade_coautores = len(coautores)
    anos_experiencia = calcular_anos_experiencia(prof)
    dados.append(
        {
            "nome": nome,
            "mestrado": mestrado,
            "doutorado": doutorado,
            "pos_doutorado": pos_doutorado,
            "anos_experiencia": anos_experiencia,
            "quantidade_producoes_bibliograficas": quantidade_producoes,
            "quantidade_coautores": quantidade_coautores,
            "producoes_bibliograficas": producoes,
            "coautores_publicacoes": coautores,
        }
    )

df = pd.DataFrame(dados)

In [30]:
df

,nome,mestrado,doutorado,pos_doutorado,anos_experiencia,quantidade_producoes_bibliograficas,quantidade_coautores,producoes_bibliograficas,coautores_publicacoes
0,Adriana Carla Damasceno,sim,sim,não,22,43,50,[{'titulo': 'Supporting the choice of the best...,"[ALBUQUERQUE, R. S., ALBUQUERQUE, RENATO, ALVE..."
1,Alan Kelon Oliveira de Moraes,sim,sim,não,20,39,62,[{'titulo': 'A family of test selection criter...,"[ALBUQUERQUE, F. C. B., ALBUQUERQUE, J. O., AL..."
2,Alisson Vasconcelos de Brito,sim,sim,sim,24,189,311,[{'titulo': 'Fault diagnosis in split-system a...,"[ABREU, GIOVANNA, ALBUQUERQUE, D. W., ALEXANDR..."
3,Alvaro Francisco de Castro Medeiros,sim,sim,não,40,53,45,[{'titulo': 'OBJECT-ORIENTED DEVELOPMENT SOFTW...,"[ALVARO, F. C., ALVARO, F. C. J., ANTAO, J., A..."
4,Ana Paula Pintado Wyse,sim,sim,sim,28,54,37,[{'titulo': 'Mathematical modeling of the perf...,"[AZEVEDO, JUAREZ DOS SANTOS, Azevedo, J. S., B..."
5,Anand Subramanian,sim,sim,não,19,179,230,[{'titulo': 'Population-based iterated local s...,"[ACIOLI, L., ALVES, R. M., ALVES, RAPHAEL MEDE..."
6,Andrei de Araujo Formiga,sim,sim,não,23,14,18,[{'titulo': 'Comparative Aspects between the C...,"[BRITO, A. V., CARVALHO JR, F. H., COSTA, E. J..."
7,Andréa Vanessa Rocha,sim,sim,não,23,25,13,[{'titulo': 'Asymptotic adjustments of Pearson...,"[Barreto-Souza, Wagner, Cordeiro, Gauss M., Cr..."
8,Antônio José Boness dos Santos,não,sim,não,24,37,45,[{'titulo': 'Mathematical modeling of the perf...,"[AZEVEDO, J. S., AZEVEDO, JUAREZ DOS SANTOS, B..."
9,Aparecido Jesuino de Souza,sim,sim,sim,47,142,42,[{'titulo': 'Multiple Traveling Waves for Dry ...,"[AKKUTLU, I. Y., ANDRADE, P. L., ARAGÃO, J. DE..."


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   nome                                 50 non-null     object
 1   mestrado                             50 non-null     object
 2   doutorado                            50 non-null     object
 3   pos_doutorado                        50 non-null     object
 4   anos_experiencia                     50 non-null     int64 
 5   quantidade_producoes_bibliograficas  50 non-null     int64 
 6   quantidade_coautores                 50 non-null     int64 
 7   producoes_bibliograficas             50 non-null     object
 8   coautores_publicacoes                50 non-null     object
dtypes: int64(3), object(6)
memory usage: 3.6+ KB


In [32]:
def normalizar_nome_citacao(nome):
    # Remove acentos
    nome = unicodedata.normalize('NFKD', nome)
    nome = ''.join([c for c in nome if not unicodedata.combining(c)])
    # Remove espaços, pontos e deixa maiúsculo
    return re.sub(r'[\s\.]', '', nome).upper()


# Cria um dicionário de mapeamento: nome_citacao -> nome principal
citacao_para_nome = {}
for prof in professores:
    nome_principal = prof.get("identificacao", {}).get("nome", "")
    for nome_citacao in prof.get("identificacao", {}).get("nomes_citacao", []):
        chave = normalizar_nome_citacao(nome_citacao)
        citacao_para_nome[chave] = nome_principal

# Função para substituir coautor por nome principal se for nome_citacao
def normalizar_coautor(coautor):

    if isinstance(coautor, str):
        chave = normalizar_nome_citacao(coautor)
        return citacao_para_nome.get(chave, coautor.strip())
    return coautor


In [33]:
coautores_df = df[["nome", "coautores_publicacoes"]].explode("coautores_publicacoes")
coautores_df = coautores_df.dropna(subset=["coautores_publicacoes"])
coautores_df = coautores_df.rename(columns={"coautores_publicacoes": "coautor"})
coautores_df["coautor"] = coautores_df["coautor"].apply(normalizar_coautor)
coautores_df

,nome,coautor
0,Adriana Carla Damasceno,"ALBUQUERQUE, R. S."
0,Adriana Carla Damasceno,"ALBUQUERQUE, RENATO"
0,Adriana Carla Damasceno,"ALVES, I. C. S."
0,Adriana Carla Damasceno,"ALVES, L. C."
0,Adriana Carla Damasceno,"ALVINO, C."
...,...,...
49,Yuska Paola Costa Aguiar,"VASCONCELOS, MICHAEL JONATHAN SILVA DE"
49,Yuska Paola Costa Aguiar,"VIEIRA, M. F. Q"
49,Yuska Paola Costa Aguiar,"VIEIRA, M. F. Q."
49,Yuska Paola Costa Aguiar,"VIEIRA, MARIA DE FÁTIMA Q."


In [34]:
coautores_lista = (
    coautores_df.groupby("nome")["coautor"]
    .unique()
    .reset_index()
    .rename(columns={"coautor": "lista_coautores_unicos"})
)
coautores_unicos = (
    coautores_df.groupby("nome")["coautor"]
    .nunique()
    .reset_index()
    .rename(columns={"coautor": "quantidade_coautores_unicos"})
    .merge(coautores_lista, on="nome")
)

coautores_unicos

,nome,quantidade_coautores_unicos,lista_coautores_unicos
0,Adriana Carla Damasceno,50,"[ALBUQUERQUE, R. S., ALBUQUERQUE, RENATO, ALVE..."
1,Alan Kelon Oliveira de Moraes,62,"[ALBUQUERQUE, F. C. B., ALBUQUERQUE, J. O., AL..."
2,Alisson Vasconcelos de Brito,310,"[ABREU, GIOVANNA, ALBUQUERQUE, D. W., ALEXANDR..."
3,Alvaro Francisco de Castro Medeiros,45,"[ALVARO, F. C., ALVARO, F. C. J., ANTAO, J., A..."
4,Ana Paula Pintado Wyse,33,"[AZEVEDO, JUAREZ DOS SANTOS, Azevedo, J. S., B..."
5,Anand Subramanian,223,"[ACIOLI, L., ALVES, R. M., ALVES, RAPHAEL MEDE..."
6,Andrei de Araujo Formiga,18,"[Alisson Vasconcelos de Brito, CARVALHO JR, F...."
7,Andréa Vanessa Rocha,13,"[Barreto-Souza, Wagner, Cordeiro, Gauss M., Cr..."
8,Antônio José Boness dos Santos,42,"[AZEVEDO, J. S., AZEVEDO, JUAREZ DOS SANTOS, B..."
9,Aparecido Jesuino de Souza,42,"[AKKUTLU, I. Y., ANDRADE, P. L., ARAGÃO, J. DE..."
